TODOs:
1. ~stopping criteria~                          (4p)   JACOB
2. ~linear systems~                             (2p)   JACOB
3. derivatives                                (6p)   LUKAS
4. line search - initial step length          (2p)   LUKAS
5. ~line search - wolfe condition~            (6p)   ???
6. Newton method with Hessian modification    (6p)
7. ~Quasi-Newton methods - initial $H_0$~      (2p) NOAH
8. ~efficient computations of quantities~       (2p)   JACOB

In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import numpy.linalg as la
from scipy import linalg

In [2]:
class Functions:
    def __init__(self):   #TODO: (3.) remove hardcoded gradient and Hessian
        """
        entries of the form:
        [
        x0 = x,
        f(x) = f,
        f'(x) = g,
        f''(x) = h
        ]
        """
        self.function_list = [  
            [
            # Rosenbrock function stationary point: (1.2,1.2)
            np.array([1.2, 1.2], dtype=np.longdouble),
            lambda x: np.array(100*(x[1]-x[0]**2)**2 + (1-x[0])**2),
            lambda x: np.array([400*x[0]**3-2*x[0]*(200*x[1]-1)-2, 200*x[1]-200*x[0]**2]),
            lambda x: np.array([[1200*x[0]**2-2*(200*x[1]-1), -400*x[0]], [-400*x[0], 200]])
            ], [
            # Rosenbrock function stationary point: (-1.2,1)
            np.array([-1.2, 1], dtype=np.longdouble),
            lambda x: np.array(100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2),
            lambda x: np.array([400 * x[0] ** 3 - 2 * x[0] * (200 * x[1] - 1) - 2, 200 * x[1] - 200 * x[0] ** 2]),
            lambda x: np.array([[1200 * x[0] ** 2 - 2 * (200 * x[1] - 1), -400 * x[0]], [-400 * x[0], 200]])
            ], [
            # Rosenbrock function stationary point: (0,1)
            np.array([0, 1], dtype=np.longdouble),
            lambda x: np.array(100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2),
            lambda x: np.array([400 * x[0] ** 3 - 2 * x[0] * (200 * x[1] - 1) - 2, 200 * x[1] - 200 * x[0] ** 2]),
            lambda x: np.array([[1200 * x[0] ** 2 - 2 * (200 * x[1] - 1), -400 * x[0]], [-400 * x[0], 200]])
            ], [
            # Rosenbrock function stationary point: (-1,0)
            np.array([-1, 0], dtype=np.longdouble),
            lambda x: np.array(100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2),
            lambda x: np.array([400 * x[0] ** 3 - 2 * x[0] * (200 * x[1] - 1) - 2, 200 * x[1] - 200 * x[0] ** 2]),
            lambda x: np.array([[1200 * x[0] ** 2 - 2 * (200 * x[1] - 1), -400 * x[0]], [-400 * x[0], 200]])
            ], [
            # Rosenbrock function stationary point: (0,-1)
            np.array([0, -1], dtype=np.longdouble),
            lambda x: np.array(100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2),
            lambda x: np.array([400 * x[0] ** 3 - 2 * x[0] * (200 * x[1] - 1) - 2, 200 * x[1] - 200 * x[0] ** 2]),
            lambda x: np.array([[1200 * x[0] ** 2 - 2 * (200 * x[1] - 1), -400 * x[0]], [-400 * x[0], 200]])
            ], [
            # second special function stationary point: (-0.2, 1.2)
            np.array([-0.2, 1.2], dtype=np.longdouble),
            lambda x: 1/10*np.array([150 * (x[0] * x[1]) ** 2 + (1/2 * x[0] + 2 * x[1] - 2) ** 2]),
            lambda x: 1/10*np.array([x[0]*(300*x[1]**2+0.5)+2*x[1]-2,
                                300*x[0]**2 *x[1]+2*x[0]+8*x[1]-8]),
            lambda x: 1/10*np.array([[300 * x[1] ** 2 + 0.5, 600 * x[0] * x[1] + 2],
                                [600 * x[0] * x[1] + 2, 300*x[0] ** 2 + 8]])
            ], [
            # second special function stationary point: (3.8, 0.1)
            np.array([3.8, 0.1], dtype=np.longdouble),
            lambda x: 1/10*np.array([150 * (x[0] * x[1]) ** 2 + (1/2 * x[0] + 2 * x[1] - 2) ** 2]),
            lambda x: 1/10*np.array([x[0]*(300*x[1]**2+0.5)+2*x[1]-2,
                                300*x[0]**2 *x[1]+2*x[0]+8*x[1]-8]),
            lambda x: 1/10*np.array([[300 * x[1] ** 2 + 0.5, 600 * x[0] * x[1] + 2],
                                [600 * x[0] * x[1] + 2, 300*x[0] ** 2 + 8]])
            ], [
            # second special function stationary point: (0, 0)
            np.array([0, 0], dtype=np.longdouble),
            lambda x: 1/10*np.array([150 * (x[0] * x[1]) ** 2 + (1/2 * x[0] + 2 * x[1] - 2) ** 2]),
            lambda x: 1/10*np.array([x[0]*(300*x[1]**2+0.5)+2*x[1]-2,
                                300*x[0]**2 *x[1]+2*x[0]+8*x[1]-8]),
            lambda x: 1/10*np.array([[300 * x[1] ** 2 + 0.5, 600 * x[0] * x[1] + 2],
                                [600 * x[0] * x[1] + 2, 300*x[0] ** 2 + 8]])
            ], [
            # second special function stationary point: (-1, 0)
            np.array([-1, 0], dtype=np.longdouble),
            lambda x: 1/10*np.array([150 * (x[0] * x[1]) ** 2 + (1/2 * x[0] + 2 * x[1] - 2) ** 2]),
            lambda x: 1/10*np.array([x[0]*(300*x[1]**2+0.5)+2*x[1]-2,
                                300*x[0]**2 *x[1]+2*x[0]+8*x[1]-8]),
            lambda x: 1/10*np.array([[300 * x[1] ** 2 + 0.5, 600 * x[0] * x[1] + 2],
                                [600 * x[0] * x[1] + 2, 300*x[0] ** 2 + 8]])
            ], [
            # second special function stationary point: (0, -1)
            np.array([0, -1], dtype=np.longdouble),
            lambda x: 1/10*np.array([150 * (x[0] * x[1]) ** 2 + (1/2 * x[0] + 2 * x[1] - 2) ** 2]),
            lambda x: 1/10*np.array([x[0]*(300*x[1]**2+0.5)+2*x[1]-2,
                                300*x[0]**2 *x[1]+2*x[0]+8*x[1]-8]),
            lambda x: 1/10*np.array([[300 * x[1] ** 2 + 0.5, 600 * x[0] * x[1] + 2],
                                [600 * x[0] * x[1] + 2, 300*x[0] ** 2 + 8]])
            ]
        ]

    def get_function(self, i):
        """
        returns a function-set with a specific index
        
        x0 = starting point
        f = function
        g = gradient of function
        h = 2nd gradient of function
        """
        x0, f, g, h = self.function_list[i]  # TODO return below implemented functions for 
        return x0, f, g, h                   # gradient and Hessian

    # TODO: edit functions for point (3.)
    def get_gradient():
        """
        should take some input and returns the gradient
        :return:
        """
        pass

    def get_hessian():
        """
        should take some input and returns the Hessian
        :return:
        """
        pass


In [3]:
def linear_solve(A, b):  # TODO: implement a linear solver (2.)   DONE
    """
    Should solve a linear system of the form A*x=b. A and b are given and x should be returned
    A: a matrix of form nxm
    b: a vector of length n
    return: a vector x of length m
    """
    try:
        x = linalg.solve(A,b)
    except linalg.LinAlgError: 
        A += 0.001
        x = linalg.solve(A,b)
    return x

In [4]:
# TODO: go over all following functions and check if the multiplication order is efficient 
# for example try to avoid matrix matrix multiplication and keep the dimensionality low (8.)   DONE

# max number of steps / amount of steps after we give up
K = 30000

# # # # # # # # # # # # # # # # # # # # # # # # # #
#                                               #
#   L I N E - S E A R C H                     #
#                                           #
# # # # # # # # # # # # # # # # # # # # # #

# line-search method for Steepest Descent & Newton
def line_search(x_0, f, g, h, a_max=10, c1=10e-4, c2=0.1, acc=0.1, eps=10e-5, method='steepest_descent'):
    """
    x_0:   starting point
    f:     function
    g:     gradient of function
    h:     2nd gradient of function
    a_max: the maximal step length that is ok for us
    c1:    parameter for wolfe condition 1
    c2:    parameter for wolfe condition 2
    acc:   accuracy for "fine-tuning" alpha
    eps: used for relative stopping criterion
    method: method
    """ 
    # start at k=0 with x_0
    k=0
    x_k = x_0
    g_0 = g(x_0)
    g_k = g_0
    
    
    
    # initialize stopping criteria (1.)
    stop_crit_1 = False
    stop_crit_2 = False
    stop_crit_3 = False
    
    
    # when the gradient is smaller than the stopping criterion, we are done
    while k <= K and not stop_crit_1 and not stop_crit_2 and not stop_crit_3 :  #TODO: stop crit (1.)   DONE
        
        if k>0:
            # prepare for next step
            x_k = x_kp1.copy()
            g_k = g(x_k)
            
        # TODO: maybe set here the inital step length to try (4.)
        # choose B_k according to method
        if method == 'steepest_descent':
            B_k = np.identity(n=len(x_k), dtype=np.longdouble)
        elif method == 'newton':     # TODO: hessian modification (5.)
            B_k = h(x_k)             # TODO: don't compute inverse (2.)              DONE
        else:
            print('input correct method')
        
        # (3.2)
        p_k = linear_solve(A=-B_k, b=g_k)  # TODO: call linear solver (2.)            DONE
        
        # find alpha that satisfies strong wolfe
        a_k = find_alpha(x_k, f, g, p_k, a_max, c1, c2, acc) # TODO: pass inital step length (4.)
        
        # (3.1)
        x_kp1 = x_k + a_k * p_k
        
        # needed for stop crit
        if k==0:
            x_1 = x_kp1
        
        # formulate stopping criteria (1.)
        stop_crit_1 = (la.norm(f(x_kp1) - f(x_k)) < eps * la.norm(f(x_1) - f(x_0)))
        stop_crit_2 = (la.norm(x_kp1 - x_k) < eps * la.norm(x_1 - x_0))
        stop_crit_3 = (la.norm(g_k) < eps * la.norm(g_0)) or (la.norm(g_k) < eps * (1+la.norm(g_0)))
        
        print('stpcrt1:', eps * la.norm(f(x_1) - f(x_0)))
        print('stpcrt3:', eps * la.norm(g_0), eps * (1+la.norm(g_0)))
        
        k += 1
    
    # returns the x for which the minimum g was found after k steps
    return x_k, g_k, k
    
    
def find_alpha(x_k, f, g, p_k, a_max=10, c1=10e-4, c2=0.1, acc=0.5):   # a_max = 100, acc = 0.01 for first
    """
    helps to select the alpha, that satisfies the strong wolfe conditions
    
    a_max: the maximal step length
    c1: see wolfe condition 1 (3.7a)
    c2: see wolfe condition 2 (3.7b)
    acc: accuracy
    """
    # lowest alpha possible (step size 0)
    a_l = np.longdouble(0)
    
    # our starting-alpha
    a_i = np.longdouble(1) # TODO: set initial step length (4.)
    
    # phi is f(x_k) in beginning
    phi_0 = f(x_k)
    
    # gradient of phi
    gphi_0 = g(x_k) @ p_k

    i = 0
    while True:
        
        # (3.3)
        phi_i = f(x_k + a_i * p_k)
        
        # (3.7a) if wolfe 1 is violated or current phi >= last phi:
        if (phi_i > phi_0 + c1 * a_i * gphi_0) or (i > 0 and phi_i >= f(x_k + a_l * p_k)):
            
            # search for alpha between too-high-alpha and lowest-alpha
            return finetune(a_l, a_i, x_k, f, g, p_k, c1, c2)
        
        # calculate gradient of current phi
        gphi_i = g(x_k + a_i * p_k) @ p_k
        
        # (3.7b) if wolfe 2 is satisfied:
        if la.norm(gphi_i) <= -c2 * gphi_0:
            
            # we are done! return found alpha
            return a_i
        
        # if not sattisfied and gradient positive:
        if gphi_i >= 0:
            
            # take a closer look, to see if we can find alpha in between
            return finetune(a_i, a_l, x_k, f, g, p_k, c1, c2)
        
        # if gradient is not positive:
        else:
            # current alpha is now lowest alpha
            a_l = a_i.copy()
            
            # make alpha a bit bigger ...
            a_i += acc * a_max
            
            # ... but not bigger than a_max!
            if a_i >= a_max:
                return a_max
        
        # prepare for next step
        i += 1

# helper for find_alpha
def finetune(a_lo, a_hi, x_k, f, g, p_k, c1=10e-4, c2=0.1):
    """
    helps to find right alpha if wolfe codndition isn't satisfied anymore
    
    a_lo: lowest alpha so far
    a_hi: highest alpha (which did not satisfy wolfe anymore)
    """
    def phi(a): return f(x_k + a * p_k)
    def gphi(a): return g(x_k + a * p_k) @ p_k
    
    # calculate phi of a_k = 0
    phi_0 = phi(0)
    gphi_0 = gphi(0)
    j = 0

    while j < 10:
        # computes new alpha between lowest and highest alpha ...
        a_j = (a_lo + a_hi) / 2
        
        # ... and the corresponding phi
        phi_j = phi(a_j)
        
        # check if new alpha violates wolfe 1
        if phi_j > phi_0 + c1 * a_j * gphi_0 or phi_j >= phi(a_lo):
            
            # if yes: we have a new highest alpha
            a_hi = a_j.copy()
        
        # if no: check wolfe 2 with new alpha
        else:
            gphi_j = gphi(a_j)
            
            # when satisfied: return the new alpha
            if la.norm(gphi_j) <= -c2 * gphi_0:
                return a_j
            
            # if not: set a_hi to a_lo and a_lo to a_j
            if gphi_j * (a_hi - a_lo) >= 0:
                a_hi = a_lo.copy()
            a_lo = a_j.copy()
            
        # repeat
        j += 1
    
    # after 10 steps, just return the until then found alpha
    return a_j


# # # # # # # # # # # # # # # # # # # # # # # # # #
#                                               #
#   C O N J U G A T E   G R A D I E N T       #
#                                           #
# # # # # # # # # # # # # # # # # # # # # #

def polak_ribiere_plus(x_0, f, g, c1=10e-4, c2=0.1, a_max=10, acc=0.1, eps=10e-5):
    """
    non-linear version of conjugate gradient:
    Fletcher-Reeves updated by Polak and Ribiere - and even the + version!
    
    c2=0.1 for conj. grad. 
    c2=0.9 for Newton
    """
    x_k = x_0
    
    f_k = f(x_k)   # f_0
    g_0 = g(x_k)
    g_k = g_0
    p_k = -g_0     # p_0
    k = 0
    
    # initialize stopping criteria (1.)
    stop_crit_1 = False
    stop_crit_2 = False
    stop_crit_3 = False
    
    while k <= K and not stop_crit_1 and not stop_crit_2 and not stop_crit_3 :  #TODO: stop crit (1.)   DONE
        
        if k>0:
            # (5.41c) / prepare for next step
            g_k = g_kp1
            x_k = x_kp1
            p_k = p_kp1
            
        # find alpha using line search
        a_k = find_alpha(x_k, f, g, p_k, c1=c1, c2=c2, a_max=a_max, acc=acc)
        
        # first line in Alg 5.4 (FR)
        x_kp1 = x_k + a_k * p_k
        
        # needed for stop crit
        if k==0:
            x_1 = x_kp1
        
        g_kp1 = g(x_kp1)
        
        # (5.44) - Polak Ribiere Method
        b_kp1 = (g_kp1.T @ (g_kp1 - g_k)) / la.norm(g_k)**2
        
        # (5.45) - Polak Ribiere +
        b_kp1 = max(b_kp1, 0)
        
        # (5.41b)
        p_kp1 = -g_kp1 + b_kp1 * p_k
        
        # formulate stopping criteria (1.)
        #stop_crit_1 = (la.norm(f(x_kp1) - f(x_k)) < eps * la.norm(f(x_1) - f(x_0)))
        #stop_crit_2 = (la.norm(x_kp1 - x_k) < eps * la.norm(x_1 - x_0))
        stop_crit_3 = (la.norm(g_k) < eps * la.norm(g_0)) #or (la.norm(g_k) < eps * (1+la.norm(g_0)))
        print(la.norm(g_k), ' <? ', eps * la.norm(g_0))
        
        k += 1
        
    return x_k, g_k, k


# # # # # # # # # # # # # # # # # # # # # # # # # #
#                                               #
#   Q U A S I - N E W T O N                   #    
#                                           #
# # # # # # # # # # # # # # # # # # # # # #

# Broyden, Fletcher, Goldfarb, Shanno (Alg 6.1) + (6.20)
def BFGS(x_0, f, g, a_max=100, acc=0.01, eps=10e-5):
    
    # initialize
    H_0 = np.identity(n=len(x_0))
    I = np.identity(n=len(x_0))
    k = 0
    
    # prepare for first step
    x_k = x_0
    H_k = H_0  # TODO: choose inital Hessian appropriate (7.)
    g_0 = g(x_0)
    g_k = g_0
    
    # initialize stopping criteria (1.)
    stop_crit_1 = False
    stop_crit_2 = False
    stop_crit_3 = False
    
    # stop after max_steps (K) or the extremum has been approximated closely enough
    while k <= K and not stop_crit_1 and not stop_crit_2 and not stop_crit_3 :  #TODO: stop crit (1.)   DONE
        
        if k>0:
            # prepare for next step
            x_k = x_kp1.copy()
            g_k = g(x_k)
        
        # (6.18) - compute search direction
        p_k = -H_k @ g_k
        
        # select alpha, that best satsfies (3.7)
        a_k = find_alpha(x_k, f, g, p_k, c1=10e-4, c2=0.9, a_max=a_max, acc=acc)
        
        # (6.3) - calculate next step
        x_kp1 = x_k + a_k * p_k
        
        # needed for stop crit
        if k==0:
            x_1 = x_kp1
        
        # (6.5) - define s_k and y_k / needed for hessian update
        s_k = x_kp1 - x_k
        y_k = g(x_kp1) - g_k
        
        # reshape from (1,) to (1,1) / [...] --> [[...]]
        s_k = s_k.reshape(s_k.shape[0], 1)
        y_k = y_k.reshape(y_k.shape[0], 1)
        
        # (6.20)
        # scale the starting matrix after the first step has been computed 
        # but before the first BFGS update is performed
        if k == 0:
            H_k = ((y_k.T @ s_k) / (y_k.T @ y_k)) * I
        
        # (6.14)
        rho_k = np.longdouble(1 / (y_k.T @ s_k))
        
        # (6.17) - (BFGS)
        H_kp1 = (I - rho_k * s_k @ y_k.T) @ H_k @ (I - rho_k * y_k @ s_k.T) + rho_k * s_k @ s_k.T
        
        # formulate stopping criteria (1.)
        #stop_crit_1 = (la.norm(f(x_kp1) - f(x_k)) < eps * la.norm(f(x_1) - f(x_0)))
        #stop_crit_2 = (la.norm(x_kp1 - x_k) < eps * la.norm(x_1 - x_0))
        stop_crit_3 = (la.norm(g_k) < eps * la.norm(g_0)) #or (la.norm(g_k) < eps * (1+la.norm(g_0)))
        print(la.norm(g_k), ' <? ', eps * la.norm(g_0))
        
        k += 1

    return x_k, g_k, k


In [5]:
# # # # # # # # # # # # # # # # # # # # # # # # # #
#                                               #
#   1 / 2   V A R .   P R O B L E M S         #
#                                           #
# # # # # # # # # # # # # # # # # # # # # #

def print_format(func_name, x_k, g_xk, k):
    
    rounding = True
    dec = 9
    
    if rounding:
        print(func_name + ":", "\t", round(x_k[0], dec), "\t\t", round(g_xk[0], dec), "\t\t", k)
        print("\t\t\t", round(x_k[1], dec), "\t\t", round(g_xk[1], dec))
    else:
        print(func_name + ":", "\t", x_k, "\t\t", g_xk, "\t\t", k)

def print_function_results(idcs):

    # get all functions, gradients and starting-points
    func = Functions()
    
    # for each function:
    for t in idcs:
        x_t, f_t, g_t, h_t = func.get_function(t)
        
        #print(g_t(x_t))
        #print(h_t(x_t))
        
        print('-' * 100)
        
        if t < 5:
            name = f"Rosenbrock {x_t}:"
        else:
            name = f"Alternative {x_t}:"
            
        print(name, "\t", "x_k", "\t\t\t", "g(x_k)", "\t\t", "k")
        print('-' * 100)
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        
        # Line Search / Steepest Descent
        x_k, g_xk, k = line_search(x_0    = x_t.copy(), 
                                   f      = f_t, 
                                   g      = g_t,
                                   h      = h_t,
                                   a_max  = 10, 
                                   c1     = 10e-4,
                                   c2     = 0.1, 
                                   acc    = 0.1, 
                                   method = 'steepest_descent'
                                  )
        print_format("Steepest Descent",  x_k, g_xk, k)
        
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        
        # Line Search / Newton's Method
        x_k, g_xk, k = line_search(x_0    = x_t.copy(), 
                                   f      = f_t, 
                                   g      = g_t,
                                   h      = h_t,
                                   a_max  = 10, 
                                   c1     = 10e-4,
                                   c2     = 0.9, 
                                   acc    = 0.1, 
                                   method = 'newton'
                                  )
        print_format("Newton's Method",  x_k, g_xk, k)
        
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        
        # Algorithm 5.4 (Polak-Ribiere +)
        x_k, g_xk, k = polak_ribiere_plus(x_0   = x_t.copy(), 
                                          f     = f_t, 
                                          g     = g_t, 
                                          c1    = 10e-4, 
                                          c2    = 0.1, 
                                          a_max = 10, 
                                          acc   = 0.1)  # c2=0.1 for conj. grad. c2=0.9 for Newton
        print_format("Conjugate Gradient",  x_k, g_xk, k)
        
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        
        # Algorithm 6.1 (BFGS) +(6.20)
        x_k, g_xk, k = BFGS(x_0   = x_t.copy(), 
                            f     = f_t, 
                            g     = g_t, 
                            a_max = 10, 
                            acc   = 0.1)
        print_format("Quasi - Newton",  x_k, g_xk, k)
        
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        
        print("\n")
        
def rosenbrock():
    idcs = [0, 1, 2, 3, 4]
    print_function_results(idcs)
    
def alternative():
    idcs = [5, 6, 7, 8, 9]
    print_function_results(idcs)

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # #
#                                               #
#   L E A S T   S Q U A R E S                 #    
#                                           #
# # # # # # # # # # # # # # # # # # # # # #


def create_data(f, q, m):
    """
    creates data points for least square problem
    :param f: function to approximate
    :param q: interval length
    :param m: number of data-points
    :return: set of data points (a,b)
    """
    
    # uniformly create m datapoints a_j ∊ [-q, q] for j=1, ..., m
    a = np.random.uniform(-q, q, m)
    a.sort()
    
    # calculate function values
    b = f(a)
    
    # return array: [[a1, a2, a3, ...,] , [b1, b2, b3, ...]]
    return np.array((a, b))


def plot_data(data, poly, ks):
    """
    plots the curves
    
    data[0]: a_j
    data[1]: b_j
    
    """
    names = ["Steepest Descent", "Newton's Method", "Conjugate Gradient", "Quasi - Newton"]
    markers = ["s", "v", "*", "x"]

    fig = plt.figure(figsize=(10, 10))
    
    # plot the "real" function
    plt.plot(data[0], data[1], "bo", label="original")
    
    # plot each approximation method on top
    for p, n, m, k in zip(poly, names, markers, ks):
        plt.plot(data[0], p(data[0]), marker=m,  label=n + "(k="+str(k)+")")

    plt.legend()
    plt.ylim(min(data[1])-0.5, max(data[1])+0.5)
    plt.show()


def second_5(function, n, q, m):
    """
    function: function to be approximated
    """
    # n = 4    # degree of polynomial
    # q = 2    # interval length of to-be-created data
    # m = 100  # number of datapoints
    
    
    # create data around given function
    data = create_data(function, q, m)
    a_j = data[0].copy()
    b_j = data[1].copy()
    
    # [1, a_j, a_j^2, a_j^3, ..., a_j^n]
    c_j = np.array([data[0] ** i for i in range(n + 1)])
    
    # definitions see task-sheet p.2
    def f(x): return np.array([(1/2)*np.sum((c_j.T @ x - data[1])**2)])
    def g(x): return c_j @ (c_j.T @ x - data[1])
    def h(x): return c_j @ c_j.T
    
    # initialize xS
    x_init = np.array([1 for i in range(n+1)], dtype=np.longdouble)
    xs = []
    ks = []
    poly = []

    algorithms =[lambda x: line_search(x.copy(), f, g, h, a_max=10, c1=10e-4, c2=0.1, acc=0.01, method='steepest_descent'),
                 lambda x: line_search(x.copy(), f, g, h, a_max=10, c1=10e-4, c2=0.9, acc=0.1, method='newton'),
                 lambda x: polak_ribiere_plus(x.copy(), f, g, c1=10e-4, c2=0.1, a_max=0.1, acc=1e-3),
                 lambda x: BFGS(x.copy(), f, g, a_max=100, acc=0.1)
                ]

    for alg in algorithms:
        x, _, k = alg(x_init)
        xs.append(x)
        ks.append(k)
        poly.append(np.poly1d(x[::-1]))
        print(x)
        
    plot_data(data, poly, ks)

In [7]:
rosenbrock()

----------------------------------------------------------------------------------------------------
Rosenbrock [1.2 1.2]: 	 x_k 			 g(x_k) 		 k
----------------------------------------------------------------------------------------------------
125.16932531574974857  <?  0.012516932531574975457
30.985565191163458865  <?  0.012516932531574975457
4.168658655035254576  <?  0.012516932531574975457
0.69469547055416311544  <?  0.012516932531574975457
0.1437250767324136676  <?  0.012516932531574975457
0.09306253958423663663  <?  0.012516932531574975457
0.4907719513361870768  <?  0.012516932531574975457
0.11917950323429381646  <?  0.012516932531574975457
0.1365281251135070918  <?  0.012516932531574975457
0.0920085023534365583  <?  0.012516932531574975457
0.522048029169203154  <?  0.012516932531574975457
0.121236516486306586534  <?  0.012516932531574975457
0.13981574586963329613  <?  0.012516932531574975457
0.09125744378489454947  <?  0.012516932531574975457
0.46238232614549956286  <?  0.01251

0.09494571020267337679  <?  0.012516932531574975457
0.089219409873946249566  <?  0.012516932531574975457
0.09110014187380008393  <?  0.012516932531574975457
0.09549499832584171225  <?  0.012516932531574975457
0.08892161597878048156  <?  0.012516932531574975457
0.09115432260542860427  <?  0.012516932531574975457
0.09043993958514602827  <?  0.012516932531574975457
0.09418457011327736771  <?  0.012516932531574975457
0.08857102254337456575  <?  0.012516932531574975457
0.09045398495987930449  <?  0.012516932531574975457
0.09482671301906200887  <?  0.012516932531574975457
0.08827552894719268581  <?  0.012516932531574975457
0.09055263385784010682  <?  0.012516932531574975457
0.08981648293826413468  <?  0.012516932531574975457
0.093582833769698566947  <?  0.012516932531574975457
0.087927155223686522116  <?  0.012516932531574975457
0.08990789340336274201  <?  0.012516932531574975457
0.09444901702558348065  <?  0.012516932531574975457
0.08763486122052274839  <?  0.012516932531574975457
0.0901119

0.049506742476033615675  <?  0.012516932531574975457
0.061856106641238988342  <?  0.012516932531574975457
0.049344981184839561808  <?  0.012516932531574975457
0.06350733971088287163  <?  0.012516932531574975457
0.049245348020167426587  <?  0.012516932531574975457
0.062278323053691977186  <?  0.012516932531574975457
0.04908410028454499999  <?  0.012516932531574975457
0.06408799493225606344  <?  0.012516932531574975457
0.04899034986014749132  <?  0.012516932531574975457
0.0597604089073859917  <?  0.012516932531574975457
0.048762653674450289817  <?  0.012516932531574975457
0.061330917867013788654  <?  0.012516932531574975457
0.048661652178778402337  <?  0.012516932531574975457
0.06319911290797640062  <?  0.012516932531574975457
0.04856982962074657404  <?  0.012516932531574975457
0.059049802531518928294  <?  0.012516932531574975457
0.048347732253992586973  <?  0.012516932531574975457
0.060688477743633301977  <?  0.012516932531574975457
0.048249183549423693393  <?  0.012516932531574975457
0

0.028189234540914918428  <?  0.012516932531574975457
0.0238384574859713812  <?  0.012516932531574975457
0.029489421550227093227  <?  0.012516932531574975457
0.023824073016400118797  <?  0.012516932531574975457
0.028022144016557824502  <?  0.012516932531574975457
0.023712449960161118107  <?  0.012516932531574975457
0.029318397277802453587  <?  0.012516932531574975457
0.023698312821622584846  <?  0.012516932531574975457
0.027868444858029808048  <?  0.012516932531574975457
0.023587548939998519441  <?  0.012516932531574975457
0.029164145170503255903  <?  0.012516932531574975457
0.02357381907165575993  <?  0.012516932531574975457
0.027727805819362330127  <?  0.012516932531574975457
0.023463741736457014878  <?  0.012516932531574975457
0.029026306466571950802  <?  0.012516932531574975457
0.023450578608014524923  <?  0.012516932531574975457
0.027599995463505254906  <?  0.012516932531574975457
0.023341020067574565185  <?  0.012516932531574975457
0.028904642939368667343  <?  0.012516932531574975

0.0155091398844694437  <?  0.012516932531574975457
0.012692492865098737327  <?  0.012516932531574975457
0.015036694802594777604  <?  0.012516932531574975457
0.012641283583238404501  <?  0.012516932531574975457
0.015348028403323747243  <?  0.012516932531574975457
0.012623598862689411041  <?  0.012516932531574975457
0.014890851235446871244  <?  0.012516932531574975457
0.012573327497013809655  <?  0.012516932531574975457
0.015193559018351461312  <?  0.012516932531574975457
0.01255533508417605893  <?  0.012516932531574975457
0.014750707845224137005  <?  0.012516932531574975457
0.012505945844214194604  <?  0.012516932531574975457
Quasi - Newton: 	 1.014027204 		 0.006895799 		 1616
			 1.028303335 		 0.010432959


----------------------------------------------------------------------------------------------------
Rosenbrock [-1.2  1. ]: 	 x_k 			 g(x_k) 		 k
----------------------------------------------------------------------------------------------------
232.8676877542265862  <?  0.02328

...

0.26737903069905623964  <?  0.020000999975001250881
0.30254539932377533007  <?  0.020000999975001250881
0.20885718380745564976  <?  0.020000999975001250881
0.2893071012340878318  <?  0.020000999975001250881
0.2043030329518208009  <?  0.020000999975001250881
0.34692577985807051543  <?  0.020000999975001250881
0.18757152295542500755  <?  0.020000999975001250881
0.42301193248052425326  <?  0.020000999975001250881
0.19671504763514246396  <?  0.020000999975001250881
0.31913579626855671524  <?  0.020000999975001250881
0.21373046326627990247  <?  0.020000999975001250881
0.23307012086447849875  <?  0.020000999975001250881
0.25977696269469069084  <?  0.020000999975001250881
0.29579087354829211497  <?  0.020000999975001250881
0.2053607093784605374  <?  0.020000999975001250881
0.2878550644140755232  <?  0.020000999975001250881
0.20247342724534035994  <?  0.020000999975001250881
0.28102516983804805005  <?  0.020000999975001250881
0.19996622145049628525  <?  0.020000999975001250881
0.27518249388604

0.07101623610134256194  <?  0.020000999975001250881
0.11092276298435976811  <?  0.020000999975001250881
0.06159470561121424116  <?  0.020000999975001250881
0.30642145512439896836  <?  0.020000999975001250881
0.06944422392032644539  <?  0.020000999975001250881
0.1077761505864118597  <?  0.020000999975001250881
0.060475419340266387503  <?  0.020000999975001250881
0.36651093565243242857  <?  0.020000999975001250881
0.072023872749303363855  <?  0.020000999975001250881
0.090811334725633877514  <?  0.020000999975001250881
0.07911608595009564374  <?  0.020000999975001250881
0.105863806659807376866  <?  0.020000999975001250881
0.05908965209848395346  <?  0.020000999975001250881
0.355566002098902456  <?  0.020000999975001250881
0.06981517159311611213  <?  0.020000999975001250881
0.08767262807995609366  <?  0.020000999975001250881
0.076767177754302088184  <?  0.020000999975001250881
0.10257073815999954653  <?  0.020000999975001250881
0.057707733242938880017  <?  0.020000999975001250881
0.3383988

0.46962984880225987618  <?  0.020000999975001250881
0.36513316125267350373  <?  0.020000999975001250881
0.5820434579349739809  <?  0.020000999975001250881
0.42204569250414085725  <?  0.020000999975001250881
0.7538934229667362268  <?  0.020000999975001250881
0.51642968492360882273  <?  0.020000999975001250881
0.38847829931088312158  <?  0.020000999975001250881
0.66230280982895230104  <?  0.020000999975001250881
0.46658285582267229534  <?  0.020000999975001250881
0.3638471692746683772  <?  0.020000999975001250881
0.59473470176031971213  <?  0.020000999975001250881
0.43143660783465876974  <?  0.020000999975001250881
0.34624081252537116353  <?  0.020000999975001250881
0.54404399970492652033  <?  0.020000999975001250881
0.40451871648970053917  <?  0.020000999975001250881
0.72493872598811003266  <?  0.020000999975001250881
0.5064888162626108038  <?  0.020000999975001250881
0.38535655499325449587  <?  0.020000999975001250881
0.67611654950758954256  <?  0.020000999975001250881
0.48082941872239

0.28231524718814245275  <?  0.020000999975001250881
0.31265586496060548396  <?  0.020000999975001250881
0.09942767816934509841  <?  0.020000999975001250881
0.11396765457695539608  <?  0.020000999975001250881
0.21480584045551931822  <?  0.020000999975001250881
0.23614916261330246178  <?  0.020000999975001250881
0.26068865904671160747  <?  0.020000999975001250881
0.28887484978095321242  <?  0.020000999975001250881
0.32107629324266124561  <?  0.020000999975001250881
0.09855418577560531708  <?  0.020000999975001250881
0.11462564528965379558  <?  0.020000999975001250881
0.22338581838901132405  <?  0.020000999975001250881
0.24674734647324993483  <?  0.020000999975001250881
0.27370554111011017267  <?  0.020000999975001250881
0.3046395961696504759  <?  0.020000999975001250881
0.097537200919009698284  <?  0.020000999975001250881
0.11287729248601928302  <?  0.020000999975001250881
0.21800352036030472485  <?  0.020000999975001250881
0.24121592886094227168  <?  0.020000999975001250881
0.2680978297

0.10092721550799259826  <?  0.020000999975001250881
0.040123357390556463886  <?  0.020000999975001250881
0.07068918861219839752  <?  0.020000999975001250881
0.08940725304288289246  <?  0.020000999975001250881
0.039174113566568066355  <?  0.020000999975001250881
0.06404161200948997862  <?  0.020000999975001250881
0.079849657721037452053  <?  0.020000999975001250881
0.10258385128519199829  <?  0.020000999975001250881
0.040002204362550944602  <?  0.020000999975001250881
0.07188626181305171576  <?  0.020000999975001250881
0.09129879588531672742  <?  0.020000999975001250881
0.03903377090713528321  <?  0.020000999975001250881
0.06518168660190801226  <?  0.020000999975001250881
0.081696116440555007426  <?  0.020000999975001250881
0.10536741872145973991  <?  0.020000999975001250881
0.039963160926618407732  <?  0.020000999975001250881
0.0736253476760547471  <?  0.020000999975001250881
0.09397435015448245412  <?  0.020000999975001250881
0.038973411239189407728  <?  0.020000999975001250881
0.0669

In [8]:
alternative()

----------------------------------------------------------------------------------------------------
Alternative [-0.2  1.2]: 	 x_k 			 g(x_k) 		 k
----------------------------------------------------------------------------------------------------
8.750182855232226867  <?  0.00087501828552322272864
0.16852611969891870547  <?  0.00087501828552322272864
0.20391416470939064636  <?  0.00087501828552322272864
0.15062761886515644362  <?  0.00087501828552322272864
0.17531137693320157085  <?  0.00087501828552322272864
0.155887400820376793  <?  0.00087501828552322272864
0.13889902505972989637  <?  0.00087501828552322272864
0.16055935555371366151  <?  0.00087501828552322272864
0.13724758239648857673  <?  0.00087501828552322272864
0.1191017126892580651  <?  0.00087501828552322272864
0.1324248027471861156  <?  0.00087501828552322272864
0.11152618866299953683  <?  0.00087501828552322272864
0.12302353048807968686  <?  0.00087501828552322272864
0.10146860507700836341  <?  0.00087501828552322272864
0

2.3547813341317854808  <?  0.00087501828552322272864
2.3524809690990993885  <?  0.00087501828552322272864
2.3501828522562055008  <?  0.00087501828552322272864
2.347886981401307208  <?  0.00087501828552322272864
2.3455933543347918437  <?  0.00087501828552322272864
2.3433019688592283043  <?  0.00087501828552322272864
2.341012822779364671  <?  0.00087501828552322272864
2.3387259139021258395  <?  0.00087501828552322272864
2.3364412400366111515  <?  0.00087501828552322272864
2.33415879899409203  <?  0.00087501828552322272864
2.331878588588009622  <?  0.00087501828552322272864
2.329600606633972446  <?  0.00087501828552322272864
2.327324850949754038  <?  0.00087501828552322272864
2.3250513193552906118  <?  0.00087501828552322272864
2.322780009672678714  <?  0.00087501828552322272864
2.3205109197261728894  <?  0.00087501828552322272864
2.3182440473421833482  <?  0.00087501828552322272864
2.3159793903492736382  <?  0.00087501828552322272864
2.3137169465781583229  <?  0.00087501828552322272864
2

1.7684299003521755349  <?  0.00087501828552322272864
1.7667024757690816629  <?  0.00087501828552322272864
1.7649767388501461897  <?  0.00087501828552322272864
1.7632526879454198262  <?  0.00087501828552322272864
1.7615303214065706643  <?  0.00087501828552322272864
1.7598096375868825739  <?  0.00087501828552322272864
1.7580906348412535924  <?  0.00087501828552322272864
1.7563733115261943235  <?  0.00087501828552322272864
1.7546576659998263328  <?  0.00087501828552322272864
1.7529436966218805474  <?  0.00087501828552322272864
1.7512314017536956566  <?  0.00087501828552322272864
1.7495207797582165148  <?  0.00087501828552322272864
1.7478118289999925443  <?  0.00087501828552322272864
1.7461045478451761432  <?  0.00087501828552322272864
1.7443989346615210909  <?  0.00087501828552322272864
1.742694987818380959  <?  0.00087501828552322272864
1.7409927056867075234  <?  0.00087501828552322272864
1.7392920866390491722  <?  0.00087501828552322272864
1.7375931290495493266  <?  0.000875018285523222

1.5021338223468959282  <?  0.00087501828552322272864
1.5006665521921235167  <?  0.00087501828552322272864
1.4992007154044016206  <?  0.00087501828552322272864
1.4977363105828418222  <?  0.00087501828552322272864
1.4962733363279267623  <?  0.00087501828552322272864
1.4948117912415087884  <?  0.00087501828552322272864
1.4933516739268086069  <?  0.00087501828552322272864
1.4918929829884139346  <?  0.00087501828552322272864
1.4904357170322781528  <?  0.00087501828552322272864
1.4889798746657189604  <?  0.00087501828552322272864
1.4875254544974170328  <?  0.00087501828552322272864
1.4860724551374146781  <?  0.00087501828552322272864
1.4846208751971144962  <?  0.00087501828552322272864
1.4831707132892780386  <?  0.00087501828552322272864
1.4817219680280244716  <?  0.00087501828552322272864
1.4802746380288292381  <?  0.00087501828552322272864
1.4788287219085227231  <?  0.00087501828552322272864
1.4773842182852889188  <?  0.00087501828552322272864
1.4759411257786640907  <?  0.00087501828552322

1.134751531824161171  <?  0.00087501828552322272864
1.13364313629469776  <?  0.00087501828552322272864
1.1325358234377831748  <?  0.00087501828552322272864
1.1314295921955383212  <?  0.00087501828552322272864
1.1303244415111182657  <?  0.00087501828552322272864
1.1292203703287112204  <?  0.00087501828552322272864
1.1281173775935375315  <?  0.00087501828552322272864
1.1270154622518486667  <?  0.00087501828552322272864
1.1259146232509262062  <?  0.00087501828552322272864
1.1248148595390808309  <?  0.00087501828552322272864
1.1237161700656513152  <?  0.00087501828552322272864
1.1226185537810035184  <?  0.00087501828552322272864
1.1215220096365293782  <?  0.00087501828552322272864
1.1204265365846459061  <?  0.00087501828552322272864
1.1193321335787941807  <?  0.00087501828552322272864
1.1182387995734383459  <?  0.00087501828552322272864
1.1171465335240646063  <?  0.00087501828552322272864
1.1160553343871802271  <?  0.00087501828552322272864
1.1149652011203125329  <?  0.00087501828552322272

...

0.00014040275380672272253  <?  8.246211251235321953e-05
0.00014026564173558437982  <?  8.246211251235321953e-05
0.00014012866356296633568  <?  8.246211251235321953e-05
0.00013999181915810839811  <?  8.246211251235321953e-05
0.00013985510839037803295  <?  8.246211251235321953e-05
0.0001397185311292701021  <?  8.246211251235321953e-05
0.00013958208724440704265  <?  8.246211251235321953e-05
0.00013944577660553860276  <?  8.246211251235321953e-05
0.00013930959908254166399  <?  8.246211251235321953e-05
0.0001391735545454202458  <?  8.246211251235321953e-05
0.0001390376428643053343  <?  8.246211251235321953e-05
0.00013890186390945469976  <?  8.246211251235321953e-05
0.00013876621755125273215  <?  8.246211251235321953e-05
0.00013863070366021060796  <?  8.246211251235321953e-05
0.00013849532210696594185  <?  8.246211251235321953e-05
0.00013836007276228236338  <?  8.246211251235321953e-05
0.00013822495549705010857  <?  8.246211251235321953e-05
0.00013808997018228507924  <?  8.246211251235321953

0.00010586351126619524639  <?  8.246211251235321953e-05
0.00010576012892706651794  <?  8.246211251235321953e-05
0.00010565684754726104472  <?  8.246211251235321953e-05
0.00010555366702818559259  <?  8.246211251235321953e-05
0.000105450587271343448024  <?  8.246211251235321953e-05
0.00010534760817833382195  <?  8.246211251235321953e-05
0.00010524472965085244282  <?  8.246211251235321953e-05
0.00010514195159069044827  <?  8.246211251235321953e-05
0.00010503927389973549119  <?  8.246211251235321953e-05
0.000104936696479970624333  <?  8.246211251235321953e-05
0.0001048342192334747368  <?  8.246211251235321953e-05
0.000104731842062422363074  <?  8.246211251235321953e-05
0.00010462956486908362148  <?  8.246211251235321953e-05
0.00010452738755582401619  <?  8.246211251235321953e-05
0.00010442531002510427901  <?  8.246211251235321953e-05
0.00010432333217948044892  <?  8.246211251235321953e-05
0.00010422145392160404736  <?  8.246211251235321953e-05
0.000104119675154221209304  <?  8.246211251235

9.1342633833460450343e-05  <?  8.246211251235321953e-05
9.125343203969819187e-05  <?  8.246211251235321953e-05
9.116431735706836746e-05  <?  8.246211251235321953e-05
9.10752897005014783e-05  <?  8.246211251235321953e-05
9.098634898501107222e-05  <?  8.246211251235321953e-05
9.089749512569391825e-05  <?  8.246211251235321953e-05
9.0808728037729228015e-05  <?  8.246211251235321953e-05
9.0720047636379605065e-05  <?  8.246211251235321953e-05
9.0631453836990006684e-05  <?  8.246211251235321953e-05
9.054294655498817726e-05  <?  8.246211251235321953e-05
9.045452570588456021e-05  <?  8.246211251235321953e-05
9.036619120527186307e-05  <?  8.246211251235321953e-05
9.0277942968825581245e-05  <?  8.246211251235321953e-05
9.0189780912303213965e-05  <?  8.246211251235321953e-05
9.010170495154487529e-05  <?  8.246211251235321953e-05
9.001371500247233396e-05  <?  8.246211251235321953e-05
8.9925810981090058486e-05  <?  8.246211251235321953e-05
8.983799280348408943e-05  <?  8.246211251235321953e-05
8.97

0.00093219269696538039487  <?  0.0001030776406404415244
0.0013283272914676447484  <?  0.0001030776406404415244
0.00083137207000286629343  <?  0.0001030776406404415244
0.001212142541169303779  <?  0.0001030776406404415244
0.00074257823248380908704  <?  0.0001030776406404415244
0.0009483197829225743818  <?  0.0001030776406404415244
0.0006272744619468894604  <?  0.0001030776406404415244
0.0009823185490321478997  <?  0.0001030776406404415244
0.0005860815665838845261  <?  0.0001030776406404415244
0.0007653660880678720577  <?  0.0001030776406404415244
0.00049267977926281871733  <?  0.0001030776406404415244
0.00079941124172130334115  <?  0.0001030776406404415244
0.00046405887016993220473  <?  0.0001030776406404415244
0.00051565446958873324957  <?  0.0001030776406404415244
0.00046909474080481761483  <?  0.0001030776406404415244
0.00042813700646413922527  <?  0.0001030776406404415244
0.0003920303768900200645  <?  0.0001030776406404415244
0.0004451519577821517132  <?  0.0001030776406404415244
0.

0.00025734867164324849282  <?  0.0001030776406404415244
0.00024414304122807170056  <?  0.0001030776406404415244
0.00023163160013199581761  <?  0.0001030776406404415244
0.00021977788017490628792  <?  0.0001030776406404415244
0.00020854733345709765444  <?  0.0001030776406404415244
0.00019790721802359343512  <?  0.0001030776406404415244
0.00018782651537194936732  <?  0.0001030776406404415244
0.0001782758284222050631  <?  0.0001030776406404415244
0.00016922730689307686984  <?  0.0001030776406404415244
0.00016065455622832178794  <?  0.0001030776406404415244
0.00015253257012054669427  <?  0.0001030776406404415244
0.00014483764919673889436  <?  0.0001030776406404415244
0.00013540918102300588435  <?  0.0001030776406404415244
0.000121199755228927872675  <?  0.0001030776406404415244
0.00010855629418754128409  <?  0.0001030776406404415244
9.795623371442515901e-05  <?  0.0001030776406404415244
Quasi - Newton: 	 3.95e-07 		 -1.2433e-05 		 139
			 0.999878446 		 -9.7164e-05


-----------------------

In [9]:
def function_6(x): return np.sin(x)
second_5(function_6, n=4, q=2, m=100)

6352.511467655351125  <?  0.63525114676553514297
17598.747578173275397  <?  0.63525114676553514297
48876.753593363314803  <?  0.63525114676553514297
135747.19551845148976  <?  0.63525114676553514297
377015.9527367100051  <?  0.63525114676553514297
1047101.1201423665291  <?  0.63525114676553514297
2908154.8354807518447  <?  0.63525114676553514297
8076932.0109001658016  <?  0.63525114676553514297
22432378.744204106657  <?  0.63525114676553514297
62302321.65516101473  <?  0.63525114676553514297
173034671.35119084003  <?  0.63525114676553514297
480575951.1714640575  <?  0.63525114676553514297
1334722359.6340374094  <?  0.63525114676553514297
3706976541.3031310218  <?  0.63525114676553514297
10295530736.099684096  <?  0.63525114676553514297
28594179638.566405075  <?  0.63525114676553514297
79415731948.205706246  <?  0.63525114676553514297
220564414177.59310691  <?  0.63525114676553514297
612582162350.81864643  <?  0.63525114676553514297
1701348365871.282357  <?  0.63525114676553514297
47252

2.6888219151538029846e+85  <?  0.63525114676553514297
7.467770125578650803e+85  <?  0.63525114676553514297
2.0740529647644969409e+86  <?  0.63525114676553514297
5.760348307876009566e+86  <?  0.63525114676553514297
1.5998440344467136794e+87  <?  0.63525114676553514297
4.443309323943454542e+87  <?  0.63525114676553514297
1.2340576533181069271e+88  <?  0.63525114676553514297
3.4273965206667509232e+88  <?  0.63525114676553514297
9.519042224885806298e+88  <?  0.63525114676553514297
2.6437607768106046077e+89  <?  0.63525114676553514297
7.3426200660498269273e+89  <?  0.63525114676553514297
2.0392945499175887638e+90  <?  0.63525114676553514297
5.6638124047195661898e+90  <?  0.63525114676553514297
1.5730327410109337736e+91  <?  0.63525114676553514297
4.368845271482625744e+91  <?  0.63525114676553514297
1.2133764611847599192e+92  <?  0.63525114676553514297
3.3699578379840686352e+92  <?  0.63525114676553514297
9.359515528017974179e+92  <?  0.63525114676553514297
2.599454804206476502e+93  <?  0.63

5.5090032241149408006e+157  <?  0.63525114676553514297
1.5300369826243691397e+158  <?  0.63525114676553514297
4.2494314723774447203e+158  <?  0.63525114676553514297
1.1802112003501273229e+159  <?  0.63525114676553514297
3.2778466636916923372e+159  <?  0.63525114676553514297
9.103691565956421673e+159  <?  0.63525114676553514297
2.5284038160200328181e+160  <?  0.63525114676553514297
7.022234673207585099e+160  <?  0.63525114676553514297
1.9503126633949099822e+161  <?  0.63525114676553514297
5.416679535805232735e+161  <?  0.63525114676553514297
1.5043955640701383091e+162  <?  0.63525114676553514297
4.1782165591183821896e+162  <?  0.63525114676553514297
1.1604324043378491256e+163  <?  0.63525114676553514297
3.2229142410020488532e+163  <?  0.63525114676553514297
8.9511256028573324294e+163  <?  0.63525114676553514297
2.4860310751928911712e+164  <?  0.63525114676553514297
6.9045512051041519875e+164  <?  0.63525114676553514297
1.9176279741477594773e+165  <?  0.63525114676553514297
5.32590307175

8.137336599795416521e+246  <?  0.63525114676553514297
2.260014276856784571e+247  <?  0.63525114676553514297
6.2768259232694252096e+247  <?  0.63525114676553514297
1.7432873798400226774e+248  <?  0.63525114676553514297
4.8417001297473186543e+248  <?  0.63525114676553514297
1.3447042878579448359e+249  <?  0.63525114676553514297
3.7346997404358281786e+249  <?  0.63525114676553514297
1.0372527459869982624e+250  <?  0.63525114676553514297
2.8808025646849320077e+250  <?  0.63525114676553514297
8.0009654819456203305e+250  <?  0.63525114676553514297
2.2221393936549262494e+251  <?  0.63525114676553514297
6.171634530827044086e+251  <?  0.63525114676553514297
1.7140721635580509096e+252  <?  0.63525114676553514297
4.760559568472792305e+252  <?  0.63525114676553514297
1.3221688028545087195e+253  <?  0.63525114676553514297
3.6721110577396523728e+253  <?  0.63525114676553514297
1.01986974668147945824e+254  <?  0.63525114676553514297
2.832524081764547631e+254  <?  0.63525114676553514297
7.866879765658

ValueError: array must not contain infs or NaNs

In [ ]:
def function_7(x): return np.cos(x)
second_5(function_7, n=4, q=2, m=100)

In [ ]:
def function_8(x): return 2**x + np.cos(0.5*x)
second_5(function_8, n=4, q=2, m=100)

In [ ]:
def function_9(x): return 5*x**2 + np.sin(x)
second_5(function_9, n=4, q=2, m=100)

In [ ]:
def function_10(x): return 2*x-5*x**3-10
second_5(function_10, n=4, q=2, m=100)